In [1]:
import pandas as pd
import requests
import urllib.request
from urllib.request import urlopen

In [22]:
#For ALL WorldBank data

def importDataWB(source):
    df = pd.read_csv(str(source))
    df = df.drop(columns = ['Country Code', 'Indicator Name','Indicator Code'])
    df = df.dropna(axis = 'columns', how = 'all')
    df = df.fillna(0)
    return df

#function starts from the end of the years provided and searches until the most recent non-null value is found
def getDataWB(country,start,df):
    countries = list(df['Country Name'])
    years = list(df.columns)[1:]
    while start >= 1960:
        row = countries.index(str(country))
        column = years.index(str(start))+1
        data = df.iloc[row,column]
        if data != 0:
            return [str(country),int(start),float(data)]
            break
        else:
            start = start - 1

#check the most recent value for each country and append it to a list of lists
#then convert and return that list as a sorted dataframe
def exportDataWB(df,start,dataName):
    countries = list(df['Country Name'])
    years = list(df.columns)[1:]
    targetList = []
    for country in countries:
        cleanData = getDataWB(str(country), start, df)
        if cleanData != None:
            targetList.append(cleanData)
    targetList = pd.DataFrame(targetList)
    targetList.columns = ['country', 'year', str(dataName)]
    targetList.sort_values('country')
    return targetList

#Hospital Beds per 1000 people by country
bedsData = importDataWB('hospitalBedsWB.csv')
bedsData = exportDataWB(bedsData, 2015, 'beds')
bedsData.to_csv('bedsData.csv')
bedsData = bedsData.drop(columns = 'year')

#Population by country
popData = importDataWB('populationWB.csv')
popData = exportDataWB(popData, 2018, 'population')
popData.to_csv('popData.csv')
popData = popData.drop(columns = 'year')

#GDP by country
gdpData = importDataWB('gdpWB.csv')
gdpData = exportDataWB(gdpData, 2018, 'gdp')
gdpData.to_csv('gdpData.csv')
gdpData = gdpData.drop(columns = 'year')

In [3]:
#Median age by country
#Source: CIA

#download raw data from text file on web
ageData = urllib.request.urlopen('https://www.cia.gov/library/publications/the-world-factbook/fields/rawdata_343.txt')
step1 = []

#clean data
for line in ageData:
    item = (line.decode('utf-8')).split(' ')
    step1.append(item)

step1 = step1[2::]
ageDataList = []

#remove extra spaces and lines and join strings that make country names
#store in a list of lists
for n in range(len(step1)):
    while '' in step1[n]:
        step1[n].remove('')
    del step1[n][0]
    if step1[n][-1] == 'est.\n':
        del step1[n][-2]
        del step1[n][-1]
    elif step1[n][-1] == '\n':
        del step1[n][-1]
    country = step1[n][:-1]
    country = ' '.join(country)
    medianAge = step1[n][-1]
    final = [country, medianAge]
    ageDataList.append(final) 

#create dataframe from list of lists
#save as csv
ageData = pd.DataFrame(ageDataList, columns = ['country', 'medianAge'])
ageData.to_csv('ageData.csv')

In [4]:
#download csv data from a url
def getDataURL(url,fileName):
    request = requests.get(url)
    response = request.content
    new_file = open(str(fileName),'wb')
    new_file.write(response)
    new_file.close()
    df = pd.read_csv(str(fileName))
    return df

#clean data so only most recent instance for each country is shown (most updated data)
def getMostRecentData(df):
    index = 0 
    limit = len(df)
    while index <= limit-2:
        country = df['country'][index]
        next_country = df['country'][index+1]
        date = df['date'][index]
        next_date = df['date'][index+1]
        if country == next_country and date < next_date:
            df.drop(index, inplace=True)
        index += 1
    return df

#COVID-19 data by country
#Source: Our World in Data, https://github.com/owid/covid-19-data/tree/master/public/data/
#direct link to download file found using inspect html elements

#download data
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
covid = getDataURL(url,'covidDataOWID.csv')

#clean up to keep only desired columns and rename to make it easier to read
covid = covid[['location','date','total_cases','total_deaths']]
covid = covid.rename(columns={'location' : 'country'})
covid['date'] = pd.to_datetime(covid['date'], infer_datetime_format=True) 

covid = getMostRecentData(covid)

#drop last row "international"
covid = covid[:-1]

#add column calculating mortality rate
covid['mortality_rate'] = covid.total_deaths / covid.total_cases

#save as csv
covid.to_csv('covidData.csv')

In [5]:
#Changes in mobility data per country
#Source: Google, https://www.google.com/covid19/mobility/
#direct link to download file found using inspect html elements

#download data
url = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=a88b56a24e1a1e25'
mobility = getDataURL(url,'globalMobilityGoogle.csv')

#clean up to keep only desired columns and rename to make them easier to read
mobility = mobility.drop(columns = ['country_region_code'])
mobility = mobility.rename(columns={'country_region' : 'country', 'retail_and_recreation_percent_change_from_baseline': 'retail_rec', 'grocery_and_pharmacy_percent_change_from_baseline' : 'grocery_pharma', 'parks_percent_change_from_baseline' : 'parks', 'transit_stations_percent_change_from_baseline' : 'transit', 'workplaces_percent_change_from_baseline' : 'work', 'residential_percent_change_from_baseline' : 'residential'})
mobility['date'] = pd.to_datetime(mobility['date'], infer_datetime_format=True) 

#drop rows that provide sub-region-level data
mobility = mobility.loc[pd.isnull(mobility).any(1),:]
mobility = mobility[mobility['sub_region_1'].isna()]
mobility = mobility[mobility['sub_region_2'].isna()]
mobility = mobility.drop(columns = ['sub_region_1', 'sub_region_2'])
mobility.reset_index(drop=True, inplace=True) #reindexing so no gaps in index values

mobility = getMostRecentData(mobility)

#add column calculating average change in outside activity
mobility['average_change_outside'] = mobility[['retail_rec','grocery_pharma', 'parks', 'transit','work']].mean(axis=1)

#clean up special characters to fit standard country name
index1 = mobility[mobility['country']== "Côte d'Ivoire"].index.values.astype(int)[0]
index2 = mobility[mobility['country']== "Réunion"].index.values.astype(int)[0]
mobility.at[index1,'country'] = "Cote d'Ivoire"
mobility.at[index2,'country'] = 'Reunion'

#save as csv
mobility.to_csv('mobilityData.csv')

/Users/alexandrabadescu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [28]:
#Merging data sources into 1 dataframe

#merge hospital beds data with population data, gdp data, and median age data
df = pd.merge(bedsData,popData)
df = df.merge(gdpData)
df = df.merge(ageData)

#calculate actual hospital beds (original data was for hospital beds per 1000 people)
#drop per 1000 people metric
df['actual_beds'] = (df.population/1000) * df.beds
df = df.drop(columns = 'beds')

#merge with covid data and drop date column
df = df.merge(covid)
df = df.drop(columns = 'date')

#merge with mobility data and drop date column
df = df.merge(mobility)
df = df.drop(columns = 'date')

#create normalized mortality rate column
df['normalized_mortality_rate'] = ((df.total_deaths)/(df.population))*100000

#export as an aggregated csv file
df.to_csv('dataMASTER.csv')

In [29]:
df

,country,population,gdp,medianAge,actual_beds,total_cases,total_deaths,mortality_rate,retail_rec,grocery_pharma,parks,transit,work,residential,average_change_outside,normalized_mortality_rate
0,Afghanistan,37172386.0,1.936297e+10,19.5,18586.193000,4967,127,0.025569,-44.0,-25.0,-21.0,-45.0,-32.0,16.0,-33.4,0.341651
1,Angola,30809762.0,1.057510e+11,15.9,24647.809600,45,2,0.044444,-33.0,-12.0,-19.0,-45.0,-30.0,19.0,-27.8,0.006491
2,United Arab Emirates,9630959.0,4.141790e+11,38.4,11557.150800,19661,203,0.010325,-53.0,-23.0,-68.0,-63.0,-44.0,30.0,-50.2,2.107786
3,Argentina,44494502.0,5.198720e+11,32.4,222472.510000,6550,319,0.048702,-73.0,-23.0,-89.0,-55.0,-40.0,21.0,-56.0,0.716943
4,Antigua and Barbuda,96286.0,1.610574e+09,32.7,365.886800,25,3,0.120000,-39.0,-26.0,-31.0,-63.0,-51.0,31.0,-42.0,3.115718
5,Australia,24982688.0,1.433900e+12,37.5,94934.214400,6964,97,0.013929,-32.0,-2.0,-25.0,-54.0,-31.0,16.0,-28.8,0.388269
6,Austria,8840521.0,4.552860e+11,44.5,67187.959600,15910,623,0.039158,-39.0,-5.0,12.0,-41.0,-38.0,12.0,-22.2,7.047096
7,Belgium,11433256.0,5.427610e+11,41.6,70886.187200,53779,8761,0.162907,-57.0,-13.0,19.0,-56.0,-52.0,22.0,-31.8,76.627340
8,Benin,11485048.0,1.035427e+10,17.0,5742.524000,327,2,0.006116,-14.0,-2.0,-1.0,-18.0,-10.0,9.0,-9.0,0.017414
9,Burkina Faso,19751535.0,1.412478e+10,17.9,7900.614000,766,51,0.066580,-17.0,-8.0,-11.0,-26.0,-7.0,11.0,-13.8,0.258208
